In [ ]:
# Bootstrap (for Colab / fresh environments)
import os
import sys
import subprocess
from pathlib import Path

REPO_URL = "https://github.com/marilevay/wildfire-simulation.git"
REPO_DIRNAME = "wildfire-simulation"

cwd = Path().resolve()
repo_dir = cwd / REPO_DIRNAME

# If the notebook is run *inside* the repo already, keep that.
if (cwd / "simulation").exists():
    repo_dir = cwd

if not repo_dir.exists():
    print(f"Cloning repo into {repo_dir} ...")
    subprocess.check_call(["git", "clone", REPO_URL, str(repo_dir)])

os.chdir(repo_dir)

# Ensure imports resolve from the cloned repo.
if str(repo_dir) not in sys.path:
    sys.path.insert(0, str(repo_dir))

print("Repo ready at:", repo_dir)

# Wildfire simulation sanity checks

## What these checks are (and are not)

- **Goal**: catch breaking changes after refactors (API changes, shape/range regressions, basic correctness invariants).
- **Not a full test suite**: these are not calibrated against real wildfire outcomes; they are structural checks.

## Core checks
- `Forest` clamps densities into `[0, 1]` and preserves array shape.
- `Forest.apply_thinning(factor)` reduces (never increases) density and preserves shape.
- `MonteCarlo.run()` returns per-run burned/affected fractions in `[0, 1]` with correct shapes and `affected >= burned`.
- `MonteCarlo.risk_map(n_runs)` returns an array shaped like the forest grid with values in `[0, 1]`.
- `DataCollector.calculate_ci95_mean` returns ordered bounds `(lo <= mean <= hi)`.

If any assertion fails, it indicates a **contract change** in the simulation API or a regression in a basic property that downstream analysis assumes.

In [2]:
# Sanity check: Forest construction + density clipping
density = np.array([[1.2, -0.1], [0.4, 0.0]], dtype=float)
forest = Forest(density)

assert forest.density.shape == (2, 2)
assert float(forest.density.max()) <= 1.0
assert float(forest.density.min()) >= 0.0
print('Forest init/clipping OK')

Forest init/clipping OK


In [ ]:
# Sanity check: thinning reduces density and preserves shape

# Specs:
# - apply_thinning(factor) returns a NEW Forest instance
# - Shape must match the input forest
# - For factor in (0, 1), density should not increase anywhere
# - For factor=0.0, density should be all zeros. For factor=1.0, should be unchanged

thin_factor = 0.5
thin = forest.apply_thinning(thin_factor)

assert thin is not forest, "apply_thinning should return a new Forest"
assert thin.density.shape == forest.density.shape, "Thinned forest should preserve shape"
assert np.all(thin.density <= forest.density + 1e-12), "Thinning should not increase density"

print("Forest thinning OK")

Forest thinning OK


In [ ]:
# Sanity check: Monte Carlo run returns valid fractions

# Specification:
# - MonteCarlo.run() returns per-run burned and affected fractions
# - Output arrays must have shape (n_runs,)
# - All fractions must lie in [0, 1]
# - Affected fraction must be >= burned fraction for each run

# Notes:
# - We use a small grid and modest n_runs to keep this notebook fast

random_number_generator = np.random.default_rng(0)

density = np.clip(random_number_generator.random((30, 30)) * 0.8, 0.0, 1.0)
forest = Forest(density)

params = FireModelParams(
    wind=(1.0, 0.0),
    wind_strength=0.3,
    density_exponent=1.2,
    base_spread=0.7,
)

n_runs = 25
mc = MonteCarlo(forest=forest, params=params, n_runs=n_runs, rng=np.random.default_rng(1))
collector = mc.run()
burned, affected = collector.convert_to_arrays()

assert burned.shape == (n_runs,), "burned_fraction should have length n_runs"
assert affected.shape == (n_runs,), "affected_fraction should have length n_runs"
assert np.all((burned >= 0.0) & (burned <= 1.0)), "burned fractions must be in [0, 1]"
assert np.all((affected >= 0.0) & (affected <= 1.0)), "affected fractions must be in [0, 1]"
assert np.all(affected + 1e-12 >= burned), "affected should be >= burned (affected includes burned + neighbors)"

print("MonteCarlo.run OK")

MonteCarlo.run OK


In [ ]:
# Sanity check: risk_map shape/range

# Specification:
# - risk_map(n_runs) returns a 2D array with same shape as forest density
# - Each cell is an estimated probability, so values must be in [0, 1]

risk_runs = 25
risk = mc.risk_map(n_runs=risk_runs)

assert risk.shape == forest.density.shape, "risk_map must match forest grid shape"
assert np.all((risk >= 0.0) & (risk <= 1.0)), "risk_map values must be in [0, 1]"

print("MonteCarlo.risk_map OK")

MonteCarlo.risk_map OK


In [ ]:
# Sanity check: CI helper returns ordered bounds

# Specification:
# - calculate_ci95_mean(samples) returns (mean, lo, hi) with lo <= mean <= hi
# - This is a structural check, not a guarantee of coverage

mean, lo, hi = DataCollector.calculate_ci95_mean(burned)
assert lo <= mean <= hi, "CI bounds must satisfy lo <= mean <= hi"

print("CI helper OK:", (mean, lo, hi))

CI helper OK: (0.07400000000000001, 0.031913258844738786, 0.11608674115526124)
